In [1]:
import pandas as pd

In [2]:
contrats = pd.read_excel('../data/Données_Assurance_S2.xlsx',sheet_name='Contrats')

In [3]:
contrats.head()

,ContratID,ClientID,Branche,Produit,DateEffet,DateEcheance,StatutContrat,StatutPaiement,PrimeAnnuelle
0,1,6915,INCENDIE,INCENDIE AUTRES,2023-09-17,2027-05-18,Suspendu,Non payé,3598
1,2,6102,ENGINEERING,RESPONSABILITE DECENNALE,2022-03-16,2024-09-18,Actif,À jour,1319
2,3,1610,INCENDIE,INCENDIE RISQUES AGRICOLES,2024-03-06,2026-06-04,Résilié,À jour,1498
3,4,4828,VIE,ASSURANCE VIE COMPLEMENT RETRAITE - HORIZON+ (AS),2021-06-27,2025-03-19,Suspendu,À jour,3255
4,5,1950,RISQUES DIVERS,R.C & INDIVIDUELLE COMPLEMENTAIRE PECHE ET PL...,2021-04-19,2026-12-05,Résilié,À jour,5332


In [17]:
contrats['StatutContrat'].value_counts()

StatutContrat
Actif       26693
Suspendu    26687
Résilié     26620
Name: count, dtype: int64

In [18]:
contrats['StatutPaiement'].value_counts()

StatutPaiement
À jour      26774
Non payé    26660
Retard      26566
Name: count, dtype: int64

* Branche: category of product
* DateEffet: date when the insurance coverage officially begins
* DateEcheance: date when the insurance coverage officially ends
* StatutContrat:
    * Actif: the policy is fully valid and operational
    * Suspendu: temporary pause in coverage (not cancelled)
    * Résilié: permanent end of coverage

* StatutPaiement: 
    * A jour: all premiums are paid on time, and the policy is fully active.
    * Non payé: a premium payment is missed or not received by the due date.
    * Late Payment: Payment is overdue but still within the grace period.

* PrimeAnnuel: refers to the total amount of money a policyholder must pay per year to maintain their insurance coverage.

dataframe shape

In [5]:
contrats.shape

(80000, 9)

checking for missing values

In [4]:
contrats.isna().sum()

ContratID         0
ClientID          0
Branche           0
Produit           0
DateEffet         0
DateEcheance      0
StatutContrat     0
StatutPaiement    0
PrimeAnnuelle     0
dtype: int64

checking for duplicate rows

In [6]:
contrats.duplicated().sum()

np.int64(0)

In [16]:
contrats['ClientID'].nunique()

14343

So in our contracts dataframe with 80k rows, we have around 14k unique clients 

Now we need to check how many contracts we got for every client

In [13]:
contrats['ClientID'].value_counts()

ClientID
2967     20
13376    17
14280    17
6259     17
12965    17
         ..
11784     1
11693     1
6372      1
4079      1
1967      1
Name: count, Length: 14343, dtype: int64

In [22]:
unique_clients = contrats.groupby('ClientID')['Produit'].agg(list).reset_index()
unique_clients.rename(columns = {'Produit':'Produits'},inplace=True)
unique_clients

,ClientID,Produits
0,1,"[RENTE EDUCATION (AS), CREDITS COMMERCIAUX (AS..."
1,2,"[AUTOMOBILE ASSURANCE FRONTIERE TEMPORAIRE, DE..."
2,3,"[SANTE ET PREVOYANCE, TOUS RISQUES MONTAGE, OM..."
3,4,"[SANTE ET PREVOYANCE, POLICE A ALIMENTER(CORPS..."
4,5,[R.C CHEF DE FAMILLE DETENTEUR D UN ANIMAL DOM...
...,...,...
14338,14396,"[POLICE ABONNEMENT(FACULTE AERIEN), PACK TOUS ..."
14339,14397,"[POLICE A ALIMENTER(CORPS DE PLAISANCE), ENGIN..."
14340,14398,[POLICE AU VOYAGE(CORPS DE CONSTRUCTION NAVAL)...
14341,14399,"[TOUS RISQUES CHANTIER, DHAMEN SFDGUI, CREDITS..."


In [27]:
unique_clients.iloc[0,1]

['RENTE EDUCATION (AS)',
 'CREDITS COMMERCIAUX (AS)',
 'POLICE AU VOYAGE(FACULTE MARITIME)',
 'POLICE A ALIMENTER(CORPS DE PECHE)']

In [36]:
clients1 = contrats.groupby('ClientID').agg({
    'Produit': list,                  # Aggregate Produit as a list              # Keep the first Branche value
    'DateEffet': 'first',             # Keep the first DateEffet
    'DateEcheance': 'first',         # Keep the first DateEcheance
    'StatutContrat': 'first',         # Keep the first StatutContrat
    'StatutPaiement': 'first',        # Keep the first StatutPaiement
    'PrimeAnnuelle': 'first'
})
clients1.rename(columns={'Produit':'Produits'},inplace=True)
clients1.reset_index()

,ClientID,Produits,DateEffet,DateEcheance,StatutContrat,StatutPaiement,PrimeAnnuelle
0,1,"[RENTE EDUCATION (AS), CREDITS COMMERCIAUX (AS...",2023-04-29,2025-05-07,Actif,Retard,3092
1,2,"[AUTOMOBILE ASSURANCE FRONTIERE TEMPORAIRE, DE...",2021-08-20,2026-01-26,Suspendu,À jour,751
2,3,"[SANTE ET PREVOYANCE, TOUS RISQUES MONTAGE, OM...",2021-03-10,2025-08-26,Actif,À jour,2370
3,4,"[SANTE ET PREVOYANCE, POLICE A ALIMENTER(CORPS...",2022-03-18,2025-04-26,Actif,À jour,4643
4,5,[R.C CHEF DE FAMILLE DETENTEUR D UN ANIMAL DOM...,2020-12-11,2025-07-12,Résilié,À jour,3269
...,...,...,...,...,...,...,...
14338,14396,"[POLICE ABONNEMENT(FACULTE AERIEN), PACK TOUS ...",2021-11-26,2025-06-13,Résilié,Non payé,3336
14339,14397,"[POLICE A ALIMENTER(CORPS DE PLAISANCE), ENGIN...",2023-03-19,2024-11-06,Résilié,Retard,2228
14340,14398,[POLICE AU VOYAGE(CORPS DE CONSTRUCTION NAVAL)...,2020-10-14,2026-10-29,Suspendu,Retard,3555
14341,14399,"[TOUS RISQUES CHANTIER, DHAMEN SFDGUI, CREDITS...",2021-03-09,2025-10-03,Résilié,Non payé,3795


Here we can add how many contracts we have per client

In [40]:
clients1['nb_contrats'] = clients1['Produits'].apply(len)

In [41]:
clients1.head()

,Produits,DateEffet,DateEcheance,StatutContrat,StatutPaiement,PrimeAnnuelle,nb_contrats
ClientID,,,,,,,
1,"[RENTE EDUCATION (AS), CREDITS COMMERCIAUX (AS...",2023-04-29,2025-05-07,Actif,Retard,3092,4
2,"[AUTOMOBILE ASSURANCE FRONTIERE TEMPORAIRE, DE...",2021-08-20,2026-01-26,Suspendu,À jour,751,6
3,"[SANTE ET PREVOYANCE, TOUS RISQUES MONTAGE, OM...",2021-03-10,2025-08-26,Actif,À jour,2370,8
4,"[SANTE ET PREVOYANCE, POLICE A ALIMENTER(CORPS...",2022-03-18,2025-04-26,Actif,À jour,4643,8
5,[R.C CHEF DE FAMILLE DETENTEUR D UN ANIMAL DOM...,2020-12-11,2025-07-12,Résilié,À jour,3269,3


In [37]:
clients = pd.read_pickle('../data/clients.pkl')

In [38]:
clients.head()

,ClientID,Nom,Prénom,CIN,DateNaissance,Sexe,SituationFamiliale,Profession,RevenusMensuels,Adresse,Email,Téléphone,WhatsApp,age
0,1,Ben Ali El Amri,Ahmed Salah,82387512,1981-12-07,F,Marié,Médecin,4041,"97 Rue Imbert - Mutuelleville, Sfax",ahmed.salah@gmail.com,21671364363,21673804239,44
1,2,Ben Ali Ben Hassen,Ahmed Salah,11727315,1998-02-28,M,Marié,Artisan,1813,"106 Rue Giraud - Centre-Ville, Mahdia",ahmed.salah@gmail.com,21623740587,21694324932,27
2,3,Ben Ali Tlili,Ahmed Salah,82121072,1980-04-13,M,Marié,Médecin,2867,"164 Rue Guichard - Lac 2, Gafsa",ahmed.salah@gmail.com,21667576259,21689757319,45
3,4,Ben Ali Belhaj,Ahmed Salah,2858647,2006-01-13,M,Marié,Ingénieur,2994,"156 Rue Colas - Centre-Ville, Sfax",ahmed.salah@gmail.com,21638268772,21694185357,19
4,5,Ben Ali Mzoughi,Ahmed Salah,52452825,1963-01-15,F,Divorcé,Étudiant,5729,"46 Rue Weber - Ezzahra, Kasserine",ahmed.salah@gmail.com,21679949500,21691349318,62


In [39]:
clients.shape

(14400, 14)

Merging Clients table with Contrats

In [42]:
merged = pd.merge(
    clients,
    clients1,
    on='ClientID',
    how='inner'
)

In [43]:
merged.shape

(14343, 21)

In [44]:
merged.head()

,ClientID,Nom,Prénom,CIN,DateNaissance,Sexe,SituationFamiliale,Profession,RevenusMensuels,Adresse,...,Téléphone,WhatsApp,age,Produits,DateEffet,DateEcheance,StatutContrat,StatutPaiement,PrimeAnnuelle,nb_contrats
0,1,Ben Ali El Amri,Ahmed Salah,82387512,1981-12-07,F,Marié,Médecin,4041,"97 Rue Imbert - Mutuelleville, Sfax",...,21671364363,21673804239,44,"[RENTE EDUCATION (AS), CREDITS COMMERCIAUX (AS...",2023-04-29,2025-05-07,Actif,Retard,3092,4
1,2,Ben Ali Ben Hassen,Ahmed Salah,11727315,1998-02-28,M,Marié,Artisan,1813,"106 Rue Giraud - Centre-Ville, Mahdia",...,21623740587,21694324932,27,"[AUTOMOBILE ASSURANCE FRONTIERE TEMPORAIRE, DE...",2021-08-20,2026-01-26,Suspendu,À jour,751,6
2,3,Ben Ali Tlili,Ahmed Salah,82121072,1980-04-13,M,Marié,Médecin,2867,"164 Rue Guichard - Lac 2, Gafsa",...,21667576259,21689757319,45,"[SANTE ET PREVOYANCE, TOUS RISQUES MONTAGE, OM...",2021-03-10,2025-08-26,Actif,À jour,2370,8
3,4,Ben Ali Belhaj,Ahmed Salah,2858647,2006-01-13,M,Marié,Ingénieur,2994,"156 Rue Colas - Centre-Ville, Sfax",...,21638268772,21694185357,19,"[SANTE ET PREVOYANCE, POLICE A ALIMENTER(CORPS...",2022-03-18,2025-04-26,Actif,À jour,4643,8
4,5,Ben Ali Mzoughi,Ahmed Salah,52452825,1963-01-15,F,Divorcé,Étudiant,5729,"46 Rue Weber - Ezzahra, Kasserine",...,21679949500,21691349318,62,[R.C CHEF DE FAMILLE DETENTEUR D UN ANIMAL DOM...,2020-12-11,2025-07-12,Résilié,À jour,3269,3


In [54]:
clients_final = merged.copy()
clients_final.head()

,ClientID,Nom,Prénom,CIN,DateNaissance,Sexe,SituationFamiliale,Profession,RevenusMensuels,Adresse,...,Téléphone,WhatsApp,age,Produits,DateEffet,DateEcheance,StatutContrat,StatutPaiement,PrimeAnnuelle,nb_contrats
0,1,Ben Ali El Amri,Ahmed Salah,82387512,1981-12-07,F,Marié,Médecin,4041,"97 Rue Imbert - Mutuelleville, Sfax",...,21671364363,21673804239,44,"[RENTE EDUCATION (AS), CREDITS COMMERCIAUX (AS...",2023-04-29,2025-05-07,Actif,Retard,3092,4
1,2,Ben Ali Ben Hassen,Ahmed Salah,11727315,1998-02-28,M,Marié,Artisan,1813,"106 Rue Giraud - Centre-Ville, Mahdia",...,21623740587,21694324932,27,"[AUTOMOBILE ASSURANCE FRONTIERE TEMPORAIRE, DE...",2021-08-20,2026-01-26,Suspendu,À jour,751,6
2,3,Ben Ali Tlili,Ahmed Salah,82121072,1980-04-13,M,Marié,Médecin,2867,"164 Rue Guichard - Lac 2, Gafsa",...,21667576259,21689757319,45,"[SANTE ET PREVOYANCE, TOUS RISQUES MONTAGE, OM...",2021-03-10,2025-08-26,Actif,À jour,2370,8
3,4,Ben Ali Belhaj,Ahmed Salah,2858647,2006-01-13,M,Marié,Ingénieur,2994,"156 Rue Colas - Centre-Ville, Sfax",...,21638268772,21694185357,19,"[SANTE ET PREVOYANCE, POLICE A ALIMENTER(CORPS...",2022-03-18,2025-04-26,Actif,À jour,4643,8
4,5,Ben Ali Mzoughi,Ahmed Salah,52452825,1963-01-15,F,Divorcé,Étudiant,5729,"46 Rue Weber - Ezzahra, Kasserine",...,21679949500,21691349318,62,[R.C CHEF DE FAMILLE DETENTEUR D UN ANIMAL DOM...,2020-12-11,2025-07-12,Résilié,À jour,3269,3


In [55]:
clients_final = clients_final.drop(columns=['DateEffet','DateEcheance','StatutContrat','StatutPaiement','PrimeAnnuelle','DateNaissance'])
clients_final.head()

,ClientID,Nom,Prénom,CIN,Sexe,SituationFamiliale,Profession,RevenusMensuels,Adresse,Email,Téléphone,WhatsApp,age,Produits,nb_contrats
0,1,Ben Ali El Amri,Ahmed Salah,82387512,F,Marié,Médecin,4041,"97 Rue Imbert - Mutuelleville, Sfax",ahmed.salah@gmail.com,21671364363,21673804239,44,"[RENTE EDUCATION (AS), CREDITS COMMERCIAUX (AS...",4
1,2,Ben Ali Ben Hassen,Ahmed Salah,11727315,M,Marié,Artisan,1813,"106 Rue Giraud - Centre-Ville, Mahdia",ahmed.salah@gmail.com,21623740587,21694324932,27,"[AUTOMOBILE ASSURANCE FRONTIERE TEMPORAIRE, DE...",6
2,3,Ben Ali Tlili,Ahmed Salah,82121072,M,Marié,Médecin,2867,"164 Rue Guichard - Lac 2, Gafsa",ahmed.salah@gmail.com,21667576259,21689757319,45,"[SANTE ET PREVOYANCE, TOUS RISQUES MONTAGE, OM...",8
3,4,Ben Ali Belhaj,Ahmed Salah,2858647,M,Marié,Ingénieur,2994,"156 Rue Colas - Centre-Ville, Sfax",ahmed.salah@gmail.com,21638268772,21694185357,19,"[SANTE ET PREVOYANCE, POLICE A ALIMENTER(CORPS...",8
4,5,Ben Ali Mzoughi,Ahmed Salah,52452825,F,Divorcé,Étudiant,5729,"46 Rue Weber - Ezzahra, Kasserine",ahmed.salah@gmail.com,21679949500,21691349318,62,[R.C CHEF DE FAMILLE DETENTEUR D UN ANIMAL DOM...,3


In [49]:
clients_final['nb_contrats'].agg(['min','max'])

min     1
max    20
Name: nb_contrats, dtype: int64

Next Step build a rule-based recommendation system that recommends 2 or 3 products for each client by adding a new column 'Recommendations', and we will see if those recommended products are present in Produits list, if not, we will append them to the recommendation list.

In [61]:
def recommander_assurances(age, situation_familiale, profession, revenus_mensuels):
    recommendations = []

    # 1. Assurance VIE (Épargne long terme)
    if (25 <= age <= 60) and (revenus_mensuels >= 3000) and (situation_familiale in ["Marié", "Divorcé"]):
        recommendations.append("AHMINI ASSURANCE")

    # 2. Assurance RISQUES DIVERS (RC Professionnelle)
    #if profession in ["Médecin", "Artisan"] and revenus_mensuels >= 2000:
        #recommendations.append("Assurance RISQUES DIVERS")
    if profession == 'Médecin' and revenus_mensuels >= 2000:
        recommendations.append('R.C MEDECIN')
    if profession == 'Artisan' and revenus_mensuels >= 2000:
        recommendations.append('RC ARTISANTS ET COMMERCANTS')

    # 3. Assurance AUTOMOBILE
    if (18 <= age <= 70) and revenus_mensuels >= 1500:
        if profession not in ["Étudiant", "Retraité"]:  # Filtre métiers peu susceptibles d'avoir une voiture
            if age <= 25:
                recommendations.append("AUTOMOBILE AMICALE")
            else:
                recommendations.append("AUTOMOBILE")

    # 4. Assurance TRANSPORT (Marchandises)
    if profession == "Commerçant" and revenus_mensuels >= 4000:
        recommendations.append("POLICE AU VOYAGE(FACULTE TERRESTRE)")

    # 5. Assurance ENGINEERING
    if profession == "Ingénieur" and revenus_mensuels >= 4500:
        recommendations.append("TOUS RISQUES CHANTIER")

    # 6. Assurance INCENDIE
    if revenus_mensuels >= 2500 and profession not in ["Étudiant"]:
        recommendations.append("INCENDIE RISQUES INDUSTRIEL")

    # 7. Assurance GROUPE-MALADIE
    if profession in ["Médecin", "Professeur", "Employé"] and revenus_mensuels >= 3500:
        recommendations.append("SANTE ET PREVOYANCE")

    return recommendations

Now we can apply it to our clients_final dataframe

In [57]:
df = clients_final.copy()

In [60]:
df['recommendations'] = df.apply(
    lambda row: recommander_assurances(
        age=row['age'],
        sexe = row['Sexe'],
        situation_familiale=row['SituationFamiliale'],
        profession=row['Profession'],
        revenus_mensuels=row['RevenusMensuels']
    ),
    axis=1
)

df[['Produits','recommendations']].head()

,Produits,recommendations
0,"[RENTE EDUCATION (AS), CREDITS COMMERCIAUX (AS...","[AHMINI ASSURANCE, R.C MEDECIN, AUTOMOBILE, IN..."
1,"[AUTOMOBILE ASSURANCE FRONTIERE TEMPORAIRE, DE...",[AUTOMOBILE]
2,"[SANTE ET PREVOYANCE, TOUS RISQUES MONTAGE, OM...","[R.C MEDECIN, AUTOMOBILE, INCENDIE RISQUES IND..."
3,"[SANTE ET PREVOYANCE, POLICE A ALIMENTER(CORPS...","[AUTOMOBILE AMICALE, INCENDIE RISQUES INDUSTRIEL]"
4,[R.C CHEF DE FAMILLE DETENTEUR D UN ANIMAL DOM...,[]


In [62]:
df['final_recommendations'] = df.apply(
    lambda row: list(set(row['recommendations']) - set(row['Produits'])),
    axis=1
)
df.head()

,ClientID,Nom,Prénom,CIN,Sexe,SituationFamiliale,Profession,RevenusMensuels,Adresse,Email,Téléphone,WhatsApp,age,Produits,nb_contrats,recommendations,final_recommendations
0,1,Ben Ali El Amri,Ahmed Salah,82387512,F,Marié,Médecin,4041,"97 Rue Imbert - Mutuelleville, Sfax",ahmed.salah@gmail.com,21671364363,21673804239,44,"[RENTE EDUCATION (AS), CREDITS COMMERCIAUX (AS...",4,"[AHMINI ASSURANCE, R.C MEDECIN, AUTOMOBILE, IN...","[INCENDIE RISQUES INDUSTRIEL, SANTE ET PREVOYA..."
1,2,Ben Ali Ben Hassen,Ahmed Salah,11727315,M,Marié,Artisan,1813,"106 Rue Giraud - Centre-Ville, Mahdia",ahmed.salah@gmail.com,21623740587,21694324932,27,"[AUTOMOBILE ASSURANCE FRONTIERE TEMPORAIRE, DE...",6,[AUTOMOBILE],[AUTOMOBILE]
2,3,Ben Ali Tlili,Ahmed Salah,82121072,M,Marié,Médecin,2867,"164 Rue Guichard - Lac 2, Gafsa",ahmed.salah@gmail.com,21667576259,21689757319,45,"[SANTE ET PREVOYANCE, TOUS RISQUES MONTAGE, OM...",8,"[R.C MEDECIN, AUTOMOBILE, INCENDIE RISQUES IND...","[INCENDIE RISQUES INDUSTRIEL, AUTOMOBILE, R.C ..."
3,4,Ben Ali Belhaj,Ahmed Salah,2858647,M,Marié,Ingénieur,2994,"156 Rue Colas - Centre-Ville, Sfax",ahmed.salah@gmail.com,21638268772,21694185357,19,"[SANTE ET PREVOYANCE, POLICE A ALIMENTER(CORPS...",8,"[AUTOMOBILE AMICALE, INCENDIE RISQUES INDUSTRIEL]","[AUTOMOBILE AMICALE, INCENDIE RISQUES INDUSTRIEL]"
4,5,Ben Ali Mzoughi,Ahmed Salah,52452825,F,Divorcé,Étudiant,5729,"46 Rue Weber - Ezzahra, Kasserine",ahmed.salah@gmail.com,21679949500,21691349318,62,[R.C CHEF DE FAMILLE DETENTEUR D UN ANIMAL DOM...,3,[],[]


Filtering the Dataframe to keep only rows where final_recommendations is not empty

In [64]:
filtered_df = df[df['final_recommendations'].apply(len)>0]

In [65]:
filtered_df.shape

(10981, 17)

In [67]:
filtered_df.head()

,ClientID,Nom,Prénom,CIN,Sexe,SituationFamiliale,Profession,RevenusMensuels,Adresse,Email,Téléphone,WhatsApp,age,Produits,nb_contrats,recommendations,final_recommendations
0,1,Ben Ali El Amri,Ahmed Salah,82387512,F,Marié,Médecin,4041,"97 Rue Imbert - Mutuelleville, Sfax",ahmed.salah@gmail.com,21671364363,21673804239,44,"[RENTE EDUCATION (AS), CREDITS COMMERCIAUX (AS...",4,"[AHMINI ASSURANCE, R.C MEDECIN, AUTOMOBILE, IN...","[INCENDIE RISQUES INDUSTRIEL, SANTE ET PREVOYA..."
1,2,Ben Ali Ben Hassen,Ahmed Salah,11727315,M,Marié,Artisan,1813,"106 Rue Giraud - Centre-Ville, Mahdia",ahmed.salah@gmail.com,21623740587,21694324932,27,"[AUTOMOBILE ASSURANCE FRONTIERE TEMPORAIRE, DE...",6,[AUTOMOBILE],[AUTOMOBILE]
2,3,Ben Ali Tlili,Ahmed Salah,82121072,M,Marié,Médecin,2867,"164 Rue Guichard - Lac 2, Gafsa",ahmed.salah@gmail.com,21667576259,21689757319,45,"[SANTE ET PREVOYANCE, TOUS RISQUES MONTAGE, OM...",8,"[R.C MEDECIN, AUTOMOBILE, INCENDIE RISQUES IND...","[INCENDIE RISQUES INDUSTRIEL, AUTOMOBILE, R.C ..."
3,4,Ben Ali Belhaj,Ahmed Salah,2858647,M,Marié,Ingénieur,2994,"156 Rue Colas - Centre-Ville, Sfax",ahmed.salah@gmail.com,21638268772,21694185357,19,"[SANTE ET PREVOYANCE, POLICE A ALIMENTER(CORPS...",8,"[AUTOMOBILE AMICALE, INCENDIE RISQUES INDUSTRIEL]","[AUTOMOBILE AMICALE, INCENDIE RISQUES INDUSTRIEL]"
5,6,Ben Ali Cherif,Ahmed Salah,77696424,M,Veuf,Médecin,3713,"157 Rue Bertrand - Mutuelleville, Nabeul",ahmed.salah@gmail.com,21650450569,21693009462,53,"[POLICE ABONNEMENT(FACULTE MARITIME), POLICE A...",4,"[R.C MEDECIN, AUTOMOBILE, INCENDIE RISQUES IND...","[INCENDIE RISQUES INDUSTRIEL, AUTOMOBILE, R.C ..."


let's test for the first row(our client is a doctor)

In [69]:
filtered_df.iloc[0,-1]

['INCENDIE RISQUES INDUSTRIEL',
 'SANTE ET PREVOYANCE',
 'AUTOMOBILE',
 'R.C MEDECIN',
 'AHMINI ASSURANCE']

Let's test for the 4th row (out client is an engineer)

In [70]:
filtered_df.iloc[3,-1]

['AUTOMOBILE AMICALE', 'INCENDIE RISQUES INDUSTRIEL']

In [74]:
filtered_df['final_recommendations'].apply(len).agg(['min','max'])

min    1
max    5
Name: final_recommendations, dtype: int64